In [ ]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')
df.head(20)

In [10]:
import numpy as np
import pandas as pd

df = pd.read_csv('../september/result/september.csv')

array = np.zeros((len(df), 10), dtype=int)

for i in range(len(df)):

    text_value = df.loc[i, 'text']
    if isinstance(text_value, str):  
        text_values = list(map(int, text_value.split()))
        length = min(len(text_values), 10)  


        array[i, :length] = text_values[:length]

with open('output_array.txt', 'w') as f:
    for row in array:
        f.write(' '.join(map(str, row)) + '\n')  



In [11]:
import pandas as pd


df = pd.read_csv('../september/result/september.csv')


vector_accuracy  = [0] * len(df)


for i in range(len(df)):

    text_values = df.loc[i, 'emotion_scores'].strip('[]').split()  


    if text_values:
        first_emotion_score = float(text_values[6])  

        if first_emotion_score < 0.3:
            vector_accuracy [i] = 0
        else:
            vector_accuracy [i] = 1

with open('vector_accuracy.txt', 'w') as f:
    for value in vector_accuracy:
        f.write(str(value) + '\n')  


In [12]:
labels = []
vector_w = [1000] * 10


for nums in array:
    temp = np.dot(nums, vector_w)
    if temp >= 0:
        labels.append(1)
    else:
        labels.append(0)

with open('label.txt', 'w') as f:
    for value in labels:
        f.write(str(value) + '\n')  

In [13]:
vector_errors = []

for i in range(len(array)):
    vector_errors.append(vector_accuracy[i] - labels[i])

print("vector_errors: ", vector_errors)

with open('vector_errors.txt', 'w') as f:
    for value in vector_errors:
        f.write(str(value) + '\n')  


vector_errors:  [0, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, 0, 0, -1, -1, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [15]:
import numpy as np

for i in range(100):
    delta = [0] * 10
    delta_w = 0

    for i in range(len(array)):
        for j in range(len(array[0])):
            delta[j] += vector_errors[i] * 0.1 * array[i][j]
            delta_w += vector_errors[i] * 0.1 

    for i in range(10):
        vector_w[i] += delta[i]

    labels = []

    for nums in array:
        temp = np.dot(nums, vector_w)
        
        temp = np.clip(temp, -500, 500)  
        label = 1 / (1 + np.exp(-temp))  
        labels.append(round(label))

    vector_errors = []

    for i in range(len(array)):
        vector_errors.append(vector_accuracy[i] - labels[i])

    # with open('vector_errors.txt', 'w') as f:
    #     for value in vector_errors:
    #         f.write(str(value) + '\n')

    with open('epoh2.txt', 'w') as f:
        TP = sum((labels[i] == 1 and vector_accuracy[i] == 1) for i in range(len(labels)))
        TN = sum((labels[i] == 0 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FP = sum((labels[i] == 1 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FN = sum((labels[i] == 0 and vector_accuracy[i] == 1) for i in range(len(labels)))

        # Расчет метрик
        N = len(array)  # Общее количество наблюдений
        accuracy = (TP + TN) / N if N > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        Fscore = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Запись результатов в файл
        f.write(f'Эпоха {i} ------------------------------------\n')
        f.write(f'accuracy = {accuracy:.4f}\n')
        f.write(f'precision = {precision:.4f}\n')
        f.write(f'recall = {recall:.4f}\n')
        f.write(f'Fscore = {Fscore:.4f}\n\n')




KeyboardInterrupt: 